In [1]:
import os
import pandas as pd
import numpy as np
import scipy.stats as spstat
import mysql.connector as mariadb
from astropy.time import Time
import matplotlib.pyplot as plt

In [2]:
#uname = os.environ["mariadb_username"]
#passwd = os.environ["mariadb_password"]

In [3]:
con = mariadb.connect(user='ztf', database='ztf')
cur = con.cursor()

In [4]:
%%time

df = pd.read_sql_query('select objectId, count(objectId) as nobs from alerts GROUP BY objectId HAVING nobs > 1', con)
obj_lst = df['objectId'].values.tolist()

CPU times: user 24.1 s, sys: 2.92 s, total: 27 s
Wall time: 20.7 s


In [5]:
def retrieve_lc(objectName):

    dflc = pd.read_sql_query("SELECT objectId, jd, magpsf, sigmapsf, diffmaglim ,ra, decl, fid, \
                             classtar, rb FROM alerts where objectId='{}' ".format(objectName), con=con)

    return dflc

In [6]:
class ZTFlc():
    '''Light curve object for NOAO formatted data'''

    def __init__(self, objId):
        '''Read in light curve data'''
        DFlc = retrieve_lc(objId)
        self.DFlc = DFlc
        self.objId = objId
        self.filename = objId
        self.filter_mag()

    def plot_multicolor_lc(self):
        '''Plot the 3 band light curve'''
        fig, ax = plt.subplots()

        now = Time.now().jd
        tc = self.DFlc['jd'] - now

        g = ax.errorbar(tc.loc[self.DFlc['fid'] == 1],
                     self.DFlc['magpsf'].loc[self.DFlc['fid'] == 1],
                     self.DFlc['sigmapsf'].loc[self.DFlc['fid'] == 1],
                     fmt = 'o', ms = 3, color = '#556B2F', label = "g")
        r = ax.errorbar(tc.loc[self.DFlc['fid'] == 2],
                     self.DFlc['magpsf'].loc[self.DFlc['fid'] == 2],
                     self.DFlc['sigmapsf'].loc[self.DFlc['fid'] == 2],
                     fmt = 'o', ms = 3, color = '#8B0000', label = "r")
        i = ax.errorbar(tc.loc[self.DFlc['fid'] == 3],
                     self.DFlc['magpsf'].loc[self.DFlc['fid'] == 3],
                     self.DFlc['sigmapsf'].loc[self.DFlc['fid'] == 3],
                     fmt = 'o', ms =3, color = '#8B008B', label = "i")

        ax.legend(fancybox = True)
        plt.gca().invert_yaxis()
        ax.set_xlabel(r"$\mathrm{Days Ago}$")
        ax.set_ylabel(r"$\mathrm{Magnitude}$")


    def filter_mag(self):
        '''Store individual passband mages as object attributes 
        and # of observations for each band.'''

        self.gMag = self.DFlc['magpsf'].loc[self.DFlc['fid'] == 1]
        self.gDmag = self.DFlc['sigmapsf'].loc[self.DFlc['fid'] == 1]

        self.rMag = self.DFlc['magpsf'].loc[self.DFlc['fid'] == 2]
        self.rDmag = self.DFlc['sigmapsf'].loc[self.DFlc['fid'] == 2]

        self.iMag = self.DFlc['magpsf'].loc[self.DFlc['fid'] == 3]
        self.iDmag = self.DFlc['sigmapsf'].loc[self.DFlc['fid'] == 3]

        self.gNobs = len(self.gMag)
        self.rNobs = len(self.rMag)
        self.iNobs = len(self.iMag)

    def real_bogus(self):
        '''Store mean real_bogus score.'''

        if not hasattr(self, 'gMag'):
            self.filter_mag()

        dummy = np.mean(self.gMag)

        self.gRB = np.mean(self.DFlc['rb'].loc[self.DFlc['fid'] == 1]) if self.gNobs > 0 else -999
        self.rRB = np.mean(self.DFlc['rb'].loc[self.DFlc['fid'] == 2]) if self.rNobs > 0 else -999
        self.iRB = np.mean(self.DFlc['rb'].loc[self.DFlc['fid'] == 3]) if self.iNobs > 0 else -999

    def star_galaxy(self):
        '''Store mean star_galaxy score.'''

        if not hasattr(self, 'gMag'):
            self.filter_mag()

        dummy = np.mean(self.gMag)

        self.gClasstar = np.mean(self.DFlc['classtar'].loc[self.DFlc['fid'] == 1]) if self.gNobs > 0 else -999
        self.rClasstar = np.mean(self.DFlc['classtar'].loc[self.DFlc['fid'] == 2]) if self.rNobs > 0 else -999
        self.iClasstar = np.mean(self.DFlc['classtar'].loc[self.DFlc['fid'] == 3]) if self.iNobs > 0 else -999

    def ra_dec(self):
        '''Store mean RA & Dec values.'''

        self.MeanRA = np.mean(self.DFlc['ra'])
        self.MeanDec = np.mean(self.DFlc['decl'])

    def weighted_mean_mag(self):
        '''Measure weighted mean mag in gri'''

        if not hasattr(self, 'gMag'):
            self.filter_mag()

        weighted_mean = lambda mag, dmag: np.sum(mag*(1./dmag)**2)/np.sum((1./dmag)**2)

        self.gMean = weighted_mean(self.gMag, self.gDmag) if self.gNobs > 0 else -999
        self.rMean = weighted_mean(self.rMag, self.rDmag) if self.rNobs > 0 else -999
        self.iMean = weighted_mean(self.iMag, self.iDmag) if self.iNobs > 0 else -999

    def sample_std_mag(self):
        '''Measure standard deviation of sample mag in gri'''

        if not hasattr(self, 'gMag'):
            self.filter_mag()

        self.gStd = np.std(self.gMag, ddof = 1) if self.gNobs > 1 else -999
        self.rStd = np.std(self.rMag, ddof = 1) if self.rNobs > 1 else -999
        self.iStd = np.std(self.iMag, ddof = 1) if self.iNobs > 1 else -999

    def normalized_amplitude(self):
        '''Measure the normalized amplitude of variations in gri'''

        if not hasattr(self, 'gMag'):
            self.filter_mag()

        if not hasattr(self, 'gMean'):
            self.weighted_mean_mag()

        normalized_amplitude = lambda mag, wMeanmag: (np.max(mag) - np.min(mag))/wMeanmag

        self.gAmp = normalized_amplitude(self.gMag, self.gMean) if self.gNobs else -999
        self.rAmp = normalized_amplitude(self.rMag, self.rMean) if self.rNobs else -999
        self.iAmp = normalized_amplitude(self.iMag, self.iMean) if self.iNobs else -999

    def normalized_MAD(self):
        '''Measure normalized Median Absolute Deviation (MAD) in gri'''

        if not hasattr(self, 'gMag'):
            self.filter_mag()

        if not hasattr(self, 'gMean'):
            self.weighted_mean_mag()

        normalized_MAD = lambda mag, wMeanmag: np.median(np.abs((mag - np.median(mag))/wMeanmag))

        self.gMAD = normalized_MAD(self.gMag, self.gMean) if self.gNobs else -999
        self.rMAD = normalized_MAD(self.rMag, self.rMean) if self.rNobs else -999
        self.iMAD = normalized_MAD(self.iMag, self.iMean) if self.iNobs else -999
        
    def normalized_beyond_1std(self):
        '''Measure fraction of mag measurements beyond 1 std'''

        if not hasattr(self, 'gMag'):
            self.filter_mag()

        if not hasattr(self, 'gMean'):
            self.weighted_mean_mag()

        beyond_1std = lambda mag, wMeanmag: sum(np.abs(mag - wMeanmag) > np.std(mag, ddof = 1))/len(mag)

        self.gBeyond = beyond_1std(self.gMag, self.gMean) if self.gNobs else -999
        self.rBeyond = beyond_1std(self.rMag, self.rMean) if self.rNobs else -999
        self.iBeyond = beyond_1std(self.iMag, self.iMean) if self.iNobs else -999

    def skew(self):
        '''Measure the skew of the mag measurements'''
        if not hasattr(self, 'gMag'):
            self.filter_mag()

        skew = lambda mag: spstat.skew(mag)

        self.gSkew = skew(self.gMag) if self.gNobs else -999
        self.rSkew = skew(self.rMag) if self.rNobs else -999
        self.iSkew = skew(self.iMag) if self.iNobs else -999

    def mean_colors(self):
        '''Measure the mean g-r, g-i, and r-i colors'''

        if not hasattr(self, 'gMag'):
            self.filter_mag()

        if not hasattr(self, 'gMean'):
            self.weighted_mean_mag()

        self.gMinusR = (self.gMean - self.rMean) if self.gMean> 0 and self.rMean > 0 else -999
        self.rMinusI = (self.rMean - self.iMean) if self.rMean> 0 and self.iMean > 0 else -999
        self.gMinusI = (self.gMean - self.iMean) if self.gMean> 0 and self.iMean > 0 else -999

In [7]:
def calculate_features(obj):
    lc = ZTFlc(obj)
    objectId = lc.objId
    lc.ra_dec()
    lc.filter_mag()
    lc.real_bogus()
    lc.star_galaxy()
    lc.weighted_mean_mag()
    lc.sample_std_mag()

    feats = (objectId, lc.MeanRA, lc.MeanDec,
             lc.gNobs,lc.rNobs,lc.iNobs,
             lc.gMean,lc.rMean,lc.iMean,
             lc.gStd,lc.rStd,lc.iStd,
             lc.gRB,lc.rRB,lc.iRB,
             lc.gClasstar,lc.rClasstar,lc.iClasstar)

    return feats

In [8]:
"""
for obj in obj_lst:
    
    feats = calculate_features(obj)

    try:
        cur.execute('INSERT INTO summary(objectId, gMeanRA, gMeanDec, gNobs, rNobs, iNobs,\
                                         gMean, rMean, iMean, gStd, rStd, iStd,\
                                         gRB, rRB, iRB, gClasstar, rClasstar, iClasstar) values {}'.format(feats))

    except mariadb.Error as error:
        print("Error: {}".format(error))

con.commit()
con.close()
"""

'\nfor obj in obj_lst:\n    \n    feats = calculate_features(obj)\n\n    try:\n        cur.execute(\'INSERT INTO summary(objectId, gMeanRA, gMeanDec, gNobs, rNobs, iNobs,                                         gMean, rMean, iMean, gStd, rStd, iStd,                                         gRB, rRB, iRB, gClasstar, rClasstar, iClasstar) values {}\'.format(feats))\n\n    except mariadb.Error as error:\n        print("Error: {}".format(error))\n\ncon.commit()\ncon.close()\n'